## <center> *Python based exercises for*</center>
# <center> Introduction to analysis and inversion of <br> SEISMOLOGICAL DATA </center>

lecturer : Nikolai Shapiro <br>
email: nikolai.shapiro@univ-grenoble-alpes.fr<br>
<br>
Main textbook for the class: [An Introduction to Seismology, Earthquakes, and Earth Structure (Stein and Wysession)](http://levee.wustl.edu/seismology/book/)

## Main Python packages used: 

- [**ObsPy**](https://github.com/obspy/obspy/wiki) : for downloading and basic analysis of seismic data

- [**NumPy**](https://numpy.org) : for mathematical functions

- [**Matplotlib**](https://matplotlib.org) : for plotting results

---

## Exercises to illustrate estimations of magnitude ans seismic moment from seismograms


---

## Step 1: Downloading waveforms and converting into displacement in microns

(ObsPy [*remove_response*](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html#obspy.core.trace.Trace.remove_response) function)

---

In [6]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
import obspy
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

#------------------ plotting mode
%matplotlib widget
#----------------------------


#-------- epicenter coordinates
evla = 38.169                # earthquake 1 to test
evlo = -117.85

#evla2 = 38.201                # earthquake 2 to test
#evlo2 = -117.745




#------------------------ selecting an FDSN datacenter 
client = Client('SCEDC')
#------------------------ selecting network, station, channel
ntw = "CI"
sta = "CGO"
chn = "HHN"




#-------------------- defining duration of the downloaded time series in sec
t_duration = 10*60


#------------ origin and location time of earthquake  -------
tstart = UTCDateTime("2020-05-15T11:03:27.000")                 # earthquake 1 to test
#tstart = UTCDateTime("2020-05-20T12:36:53.000")                 # earthquake 2 to test


#------------- onset before the earthquake starting time (sec)
tshift = -100


#-----------  getting station inventory (to know coordinates)
inv = client.get_stations(network=ntw, station=sta,  starttime=tstart+tshift, endtime=tstart+tshift + t_duration, channel=chn, includeavailability=True)


#------------  getting waveforms
st1 = client.get_waveforms(ntw, sta, "*", chn, tstart+tshift, tstart+tshift + t_duration, attach_response=True)



#-------------- extracting a trace from the stream
s1 = st1[0]

#------------ detrending time series
s1.detrend()



#--------- adding coordinates to the trace header
s1.stats.stla = inv[0][0].latitude
s1.stats.stlo = inv[0][0].longitude
s1.stats.evla = evla
s1.stats.evlo = evlo
dst = obspy.geodetics.base.gps2dist_azimuth(s1.stats.stla,s1.stats.stlo,s1.stats.evla,s1.stats.evlo)
s1.stats.dist = dst[0]/1000.


#----------- printing information in the header (trace.stats)
print("network: ", s1.stats.network)
print("station: ", s1.stats.station)

print("start time: ", s1.stats.starttime)


print("station latitude: ", s1.stats.stla)
print("station longitude: ", s1.stats.stlo)

print("event latitude: ", s1.stats.evla)
print("event longitude: ", s1.stats.evlo)

print("epicentral distance (km): ", s1.stats.dist)

print("component: ", s1.stats.channel)
print("discretization time step: ", s1.stats.delta)
print("number of samples: ", s1.stats.npts)




#------------------ correcting for instrument response
pre_filt = (0.0003, 0.005, 30.0, 35.0)                   # defining spectral band
st1.remove_response(output='DISP', pre_filt=pre_filt)

#------------------------- converting to microns
s1.data = s1.data*1e6

#------------- tapering the trace to remove edge effects
s1.taper(0.01)

#-------- plotting corrected displacement seismogram
dt = s1.stats.delta
npts = s1.stats.npts
time = dt*(np.linspace(1,npts,npts)-1) + tshift

plt.figure()
plt.plot(time,s1.data)
plt.xlabel('time (s)')
plt.ylabel('ground displacement (microns)')
plt.title(s1.stats.station + ' ' + s1.stats.channel + '   ' + str(s1.stats.starttime))
plt.show()

network:  CI
station:  CGO
start time:  2020-05-15T11:01:46.998300Z
station latitude:  36.5504
station longitude:  -117.80295
event latitude:  38.169
event longitude:  -117.85
epicentral distance (km):  179.687717985902
component:  HHN
discretization time step:  0.01
number of samples:  60001


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

## Step 2: Filtering

ObsPy [*filter*](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.filter.html) function.


### This high-pass filering is applied to obtain a signal approximately corresponding to **Wood-Anderson** seismometer
---

In [7]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
from obspy import read

#------------------ plotting mode
%matplotlib widget
#----------------------------

st1hf = st1.copy()

st1hf.filter("highpass", freq=0.8, corners=4, zerophase=True)

s1hf = st1hf[0]


plt.figure()
plt.plot(time,s1hf.data)
plt.xlabel('time (s)')
plt.ylabel('filtered ground displacement (microns)')
plt.title(s1hf.stats.station + ' ' + s1hf.stats.channel + '   ' + str(s1hf.stats.starttime))
plt.show()


#----------- printing information in the header (trace.stats)
print("network: ", s1hf.stats.network)
print("station: ", s1hf.stats.station)

print("start time: ", s1hf.stats.starttime)


print("station latitude: ", s1hf.stats.stla)
print("station longitude: ", s1hf.stats.stlo)

print("event latitude: ", s1hf.stats.evla)
print("event longitude: ", s1hf.stats.evlo)

print("epicentral distance (km): ", s1hf.stats.dist)

print("component: ", s1hf.stats.channel)
print("discretization time step: ", s1hf.stats.delta)
print("number of samples: ", s1hf.stats.npts)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

network:  CI
station:  CGO
start time:  2020-05-15T11:01:46.998300Z
station latitude:  36.5504
station longitude:  -117.80295
event latitude:  38.169
event longitude:  -117.85
epicentral distance (km):  179.687717985902
component:  HHN
discretization time step:  0.01
number of samples:  60001


---

## Step 3: Estimating magnitudes $M_s$ and $M_l$


$$M_s = \log A_{20} + 1.66 \log \Delta + 2.0$$

where $A_{20}$ is the maximum displacement ampitude (in microns) measured at period ~20 s and $\Delta$ is the epicentral distance in degrees.

$A_{20}$ can be **approximately** estimated as the maximum amplitude of unfiltered displacement seismogram


$$M_l = \log A + 2.76 \log \Delta - 2.48$$

where $A$ is the maximum displacement ampitude (in microns) recorded by a **Wood-Anderson** seismometer and $\Delta$ is the epicentral distance in kilometers.

$A$ can be **approximately** estimated as the maximum amplitude of high-pass filtered displacement seismogram

---

In [8]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np
#------------------------ importing ObsPy functions
from obspy import read

#------------------ plotting mode
%matplotlib widget
#----------------------------

#----------- printing information in the header (trace.stats)
print("network: ", s1.stats.network)
print("station: ", s1.stats.station)

print("start time: ", s1.stats.starttime)


print("station latitude: ", s1.stats.stla)
print("station longitude: ", s1.stats.stlo)

print("event latitude: ", s1.stats.evla)
print("event longitude: ", s1.stats.evlo)

print("epicentral distance (km): ", s1.stats.dist)

print("component: ", s1.stats.channel)
print("discretization time step: ", s1.stats.delta)
print("number of samples: ", s1.stats.npts)
print(' ')

#-------------------------- Ms from long period seismogram
Alp = max(np.abs(s1.data))
Ms = np.log10(Alp) + 1.66*np.log10(s1.stats.dist/111.11) + 2.0
print("long period Amplitude: (microns)", Alp)
print("Ms estimation: ", Ms)
print(' ')

#-------------------------- Ml from filtered seismogram
Ahf = max(np.abs(s1hf.data))
Ml = np.log10(Ahf) + 2.76*np.log10(s1.stats.dist) - 2.48
print("high frequency Amplitude: (microns)", Ahf)
print("Ml estimation: ", Ml)



network:  CI
station:  CGO
start time:  2020-05-15T11:01:46.998300Z
station latitude:  36.5504
station longitude:  -117.80295
event latitude:  38.169
event longitude:  -117.85
epicentral distance (km):  179.687717985902
component:  HHN
discretization time step:  0.01
number of samples:  60001
 
long period Amplitude: (microns) 4114.98413745
Ms estimation:  5.96091847318
 
high frequency Amplitude: (microns) 310.645748262
Ml estimation:  6.23473617921


---

## Step 3: Computing displacement amplitude spectra. Estimating seismic moment and [moment magnitude](https://en.wikipedia.org/wiki/Moment_magnitude_scale) $M_w$ 

We assume that the dominant part of the signal is the S-wave (**!!! approximation 1**).

Then, we use an expression for the S-wave far-field term in a homogeneous space (**!!! approximation 2**) and ignore the radiation paatern (**!!! approximation3**).

Amplitude Fourier spectra is computed and multiplied by the normalization factor of the far-field S-waves (Aki & Richards, eq. 4.29) :

$$norm_S = 4 \pi \rho \beta^3 \Delta$$

where $\rho$ is the densisty, $\beta$ is the S-wave speed, and $\Delta$ is the epicentral distance (all in international units).

If a "plateau" is observed in the low-frequency end of the spectra, we define its limits $[f_{min},f_{max}]$ and estimate the seismic moment $M_0$ as average within these limits.

The moment magnitude is than estimated with the Hanks and Kanamori (1979) formula:

$$M_w = (\log M_0 - 9.05)/1.5$$

---

In [9]:
#------------------------ importing basic packages
import matplotlib.pyplot as plt
import numpy as np

#------------------ plotting mode
%matplotlib widget
#----------------------------

#-----------------------------------------------
# function to compute Fourier spectra
#-----------------------------------------------
def signal_fft1d(sig,dt):
    npt = np.size(sig)
    spe = np.fft.fft(sig)
    freq = np.fft.fftfreq(npt,dt)
    sp_amp = np.sqrt(spe.real**2+spe.imag**2)
    sp_pha = np.arctan2(spe.imag, spe.real)
    npt_spe = int(npt/2)
    return npt_spe, dt*sp_amp[0:npt_spe],sp_pha[0:npt_spe],freq[0:npt_spe]


#----------- defining S-wave far field normalization factor
rho = 2800
Vs = 3600
moment_norm = 4*np.pi*rho*Vs**3*s1.stats.dist*1000.

#-------------------------------- renormalizing the signal (after re-converting to meters)
sig1 = moment_norm*s1.data/1e6


#----------- Fourier transform
nspe, spamp, sppha, fr = signal_fft1d(sig1,s1.stats.delta)


#----------- defining limits of spectral plateau
fmin = 0.05
fmax = .12


#------------ estimating seismic mopment as average
M0 = 0
nM0 = 0
for i in range(0,nspe):
    if (fr[i]>=fmin) & (fr[i]<=fmax):
        M0 = M0 + spamp[i]
        nM0 = nM0+1
M0 = M0/nM0


#---------------------------- computing moment magnitude
Mw =(np.log10(M0)-9.05)/1.5


#----------- printing information in the header (trace.stats)
print("network: ", s1.stats.network)
print("station: ", s1.stats.station)

print("start time: ", s1.stats.starttime)


print("station latitude: ", s1.stats.stla)
print("station longitude: ", s1.stats.stlo)

print("event latitude: ", s1.stats.evla)
print("event longitude: ", s1.stats.evlo)

print("epicentral distance (km): ", s1.stats.dist)

print("component: ", s1.stats.channel)
print("discretization time step: ", s1.stats.delta)
print("number of samples: ", s1.stats.npts)

plt.figure()
plt.loglog(fr,spamp)
plt.loglog([.12,12],[moment_norm*.03,moment_norm*.000003],'k:')
plt.loglog([fmin,fmax],[M0,M0],'r:')
plt.xlim(.05,10)
plt.ylim(7.e14,5.e19)
plt.xlabel('frequency(Hz)')
plt.ylabel('seismic moment (N m)')
plt.title(s1.stats.station + ' ' + s1.stats.channel + '   ' + str(s1.stats.starttime))
plt.show()


print("seismic moment (N m): ", M0)
print("moment magnitude Mw: ", Mw)





network:  CI
station:  CGO
start time:  2020-05-15T11:01:46.998300Z
station latitude:  36.5504
station longitude:  -117.80295
event latitude:  38.169
event longitude:  -117.85
epicentral distance (km):  179.687717985902
component:  HHN
discretization time step:  0.01
number of samples:  60001


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

seismic moment (N m):  6.52138441475e+18
moment magnitude Mw:  6.50955986749
